In [3]:
from mesa import Agent, Model
from mesa.space import NetworkGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import networkx as nx
import graph_init
import pandas as pd
import station as st

import network_example as ne
import numpy as np
import pathfinding as pf

Could not import SolaraViz. If you need it, install with 'pip install --pre mesa[viz]'


In [4]:
class Commuter(Agent):
    def __init__(self, model, current_pos, distance_left, intermediate_node, destination):
        super().__init__(model)
        self.destination = destination # The agent's destination
        self.intermediate_node = intermediate_node # The agent's intermediate node needed to reach the destination
        self.distance_left = distance_left # The distance left to the intermediate node, in terms of time
        self.current_pos = current_pos # The agent's current position 
        self.biking = False # Whether the agent is currently biking
        self.stopwatch = 0 # The agent's stopwatch, used to measure time spent completing a trip
        self.walking_time = pf.walk_cost(self.model.grid.G, self.current_pos, self.destination, walk_multiplier=self.model.walking_multiplier) # The time it takes to walk to the destination directly from the initial destination
        self.time_saved = [[self.walking_time]] # List of lists, where each list contains the time needed to walk directly, and time actually spent walking/biking

        assert self.model.grid.G.nodes[self.current_pos]['type'] == 'destination', "Agent must start at a destination"
        
    def step(self):
        # update the agent's position and stopwatch for current trip
        self.current_pos = self.model.get_node(self)
        self.stopwatch += 1

        # If the agent has reached the intermediate node, set new start node
        if self.distance_left <= 0:
            # If the agent has reached the intermediate node, set new start node
            self.current_pos = self.intermediate_node
            self.model.grid.move_agent(self, self.current_pos)

            # Return bike if the agent is at the intended station and there are spots available
            if self.biking and self.model.grid.G.nodes[self.current_pos]['type'] == 'station' and self.model.grid.G.nodes[self.current_pos]['data'].get_spot_availability():
                self.model.grid.G.nodes[self.current_pos]['data'].return_bike()
                self.biking = False

            # If the agent has reached the destination, set new destination
            # Else, the agent will rent a bike if it is at a station and there are bikes available
            if self.current_pos == self.destination:
                self.time_saved[-1].append(self.stopwatch)
                self.stopwatch = 0
                self.destination = self.model.sample_destination(self.current_pos)
                self.walking_time = pf.walk_cost(self.model.grid.G, self.current_pos, self.destination, walk_multiplier=self.model.walking_multiplier)
                self.time_saved.append([self.walking_time])
            elif not(self.biking) and self.model.grid.G.nodes[self.current_pos]['type'] == 'station' and self.model.grid.G.nodes[self.current_pos]['data'].get_bike_availability():
                self.model.grid.G.nodes[self.current_pos]['data'].rent_bike()
                self.biking = True

            # Set new intermediate node
            if self.model.grid.G.nodes[self.current_pos]['type'] == 'destination':
                path = pf.pathfind(self.model.grid.G, self.current_pos, self.destination, False, walk_multiplier=self.model.walking_multiplier)
            else:
                bike = self.model.grid.G.nodes[self.current_pos]['data'].get_bike_availability()
                path = pf.pathfind(self.model.grid.G, self.current_pos, self.destination, bike, walk_multiplier=self.model.walking_multiplier)

            self.intermediate_node = path[0]

            if path[1]:
                self.distance_left = self.model.grid.G[self.current_pos][self.intermediate_node]['weight']
            else:
                self.distance_left = self.model.grid.G[self.current_pos][self.intermediate_node]['weight'] * self.model.walking_multiplier
        else:
            self.distance_left -= 1        

    
    def get_agent_position(self):
        return self.current_pos
    
    def get_distance_left(self):
        return self.distance_left
    
    def get_agent_destination(self):
        return self.destination
    
    def get_intermediate_node(self):
        return self.intermediate_node
    
    def get_stopwatch(self):
        return self.stopwatch


class MyModel(Model):
    def __init__(self, n_agents, seed=None):
        # Initialize the model, set up random seeds for mesa and numpy
        super().__init__(seed=seed)
        self.rng = np.random.default_rng(seed)

        # graph initialization and time multiplier for walking
        g = ne.basic_graph()[0]
        self.walking_multiplier = 3
        self.grid = NetworkGrid(g)
        
        # data collector
        self.datacollector = DataCollector(
            agent_reporters={"Position": lambda agent: agent.get_agent_position(),
                             "Distance_Left": lambda agent: agent.get_distance_left(),
                             "Intermediate_Node": lambda agent: agent.get_intermediate_node(),
                             "Destination": lambda agent: agent.get_agent_destination(),
                             "Stopwatch": lambda agent: agent.get_stopwatch()})
        
        # storing all stations and destination node indices
        self.stations = pf.get_stations(self.grid.G)
        self.destinations = pf.get_destinations(self.grid.G)

        # destination weights for sampling
        self.destination_w = ne.basic_weights()

        # Create agents and place them on the grid
        for i in range(1, n_agents + 1):
            node_id = self.random.choice(self.destinations)
            destination_node = self.sample_destination(node_id)
            path = pf.pathfind(self.grid.G, node_id, destination_node, False, walk_multiplier=self.walking_multiplier)
            intermediate_node = path[0]
            distance_left = self.grid.G[node_id][intermediate_node]['weight'] * self.walking_multiplier

            commuter = Commuter(self, current_pos=node_id, distance_left=distance_left, intermediate_node=intermediate_node, destination=destination_node)

            self.grid.place_agent(commuter, node_id)
            
        self.datacollector.collect(self)
    
    def get_node(self, agent):
        """
        Returns the node ID of the given agent if it is found in the grid

        Parameters
        ----------
        agent : Agent
            The agent whose node ID is to be found

        Returns
        -------
        node_id : int
            The node ID of the given agent
        """
        # Iterate over all node IDs in the graph
        for node_id in self.grid.G.nodes():
            # Get the agents at the current node
            agents_at_node = self.grid.get_cell_list_contents([node_id])
            # If the given agent is found at this node, return the node ID
            if agent in agents_at_node:
                return node_id
        return None  # If the agent is not found in any node

    def sample_destination(self, current_pos):
        """
        Sample a destination for an agent to travel to

        Parameters
        ----------
        current_pos : int
            The current position index of the agent
        
        Returns
        -------
        destination: int
            The destination node index
        """
        while True:
            destination = self.rng.choice(self.destinations, p=ne.basic_weights())
            if destination != current_pos:
                return destination
            
    def assign_bikes(self, bike_counts):
        """
            For each station, assign a number of bikes to the station

            Parameters
            ----------
            bike_counts : dict
                A dictionary where the keys are station indices and the values are the number of bikes to be assigned to the station

            Returns
            -------
            None
        """
        for station in self.stations:
            assert station in bike_counts, "All stations must be assigned a number of bikes"
            self.grid.G.nodes[station]['data'].assign_bikes(bike_counts[station])

    def step(self):
        self.agents.shuffle_do("step")
        self.datacollector.collect(self)

In [20]:
model = MyModel(1, seed=1)
for i in range(1000):
    print(model.agents[0].get_agent_position(), model.agents[0].get_distance_left(), model.agents[0].get_agent_destination(), model.agents[0].get_intermediate_node())
    model.step()

agent_data = model.datacollector.get_agent_vars_dataframe()
agent_id = 8
agent_specific_data = agent_data[agent_data.index.get_level_values('AgentID') == agent_id]

# Print the filtered DataFrame
# print(agent_specific_data)

print(model.agents[0].time_saved)

5 15.0 6 1
5 14.0 6 1
5 13.0 6 1
5 12.0 6 1
5 11.0 6 1
5 10.0 6 1
5 9.0 6 1
5 8.0 6 1
5 7.0 6 1
5 6.0 6 1
5 5.0 6 1
5 4.0 6 1
5 3.0 6 1
5 2.0 6 1
5 1.0 6 1
5 0.0 6 1
1 8.0 6 2
1 7.0 6 2
1 6.0 6 2
1 5.0 6 2
1 4.0 6 2
1 3.0 6 2
1 2.0 6 2
1 1.0 6 2
1 0.0 6 2
2 15.0 6 6
2 14.0 6 6
2 13.0 6 6
2 12.0 6 6
2 11.0 6 6
2 10.0 6 6
2 9.0 6 6
2 8.0 6 6
2 7.0 6 6
2 6.0 6 6
2 5.0 6 6
2 4.0 6 6
2 3.0 6 6
2 2.0 6 6
2 1.0 6 6
2 0.0 6 6
6 36.0 7 7
6 35.0 7 7
6 34.0 7 7
6 33.0 7 7
6 32.0 7 7
6 31.0 7 7
6 30.0 7 7
6 29.0 7 7
6 28.0 7 7
6 27.0 7 7
6 26.0 7 7
6 25.0 7 7
6 24.0 7 7
6 23.0 7 7
6 22.0 7 7
6 21.0 7 7
6 20.0 7 7
6 19.0 7 7
6 18.0 7 7
6 17.0 7 7
6 16.0 7 7
6 15.0 7 7
6 14.0 7 7
6 13.0 7 7
6 12.0 7 7
6 11.0 7 7
6 10.0 7 7
6 9.0 7 7
6 8.0 7 7
6 7.0 7 7
6 6.0 7 7
6 5.0 7 7
6 4.0 7 7
6 3.0 7 7
6 2.0 7 7
6 1.0 7 7
6 0.0 7 7
7 48.0 4 4
7 47.0 4 4
7 46.0 4 4
7 45.0 4 4
7 44.0 4 4
7 43.0 4 4
7 42.0 4 4
7 41.0 4 4
7 40.0 4 4
7 39.0 4 4
7 38.0 4 4
7 37.0 4 4
7 36.0 4 4
7 35.0 4 4
7 34.0 4 4
7 33.0 4 4
7 32.